## Import statements for Dataframe manipulation

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download Ballot dataframe from CSV file

In [3]:
filename = 'TakomaPark_11082022_CityCouncilWard2'

data = pd.read_csv('/content/drive/MyDrive/RCV Voting Method Research/Voting Ballots/' + filename + '.csv')
data.head()

,ballotID,ward,rank1,rank2
0,3,200,Cindy Dyballa,skipped
1,6,200,Cindy Dyballa,skipped
2,11,200,Cindy Dyballa,skipped
3,12,200,Cindy Dyballa,skipped
4,15,200,Cindy Dyballa,skipped


# Remove irrelevant data

Remove nan

In [4]:
#data = data.dropna()

Find irrelevant attributes

In [5]:
irrelevantAttributes = []

attributes = data.columns

pattern = re.compile("^[rR][aA][nN][kK]")

length = len(attributes)

for i in range(1, length + 1):
    if re.search(pattern, attributes[length - i]) == None:
      irrelevantAttributes.append(attributes[length - i])

Remove attributes

In [6]:
data = data.drop(columns = irrelevantAttributes)
attributes = data.columns

attributes

Index(['rank1', 'rank2'], dtype='object')

# Determine Candidates

In [7]:
print(data.rank1.unique())

candNum = data.shape[1]

['Cindy Dyballa' 'WRITE-IN14' 'WRITE-IN25' 'WRITE-IN29' 'WRITE-IN60'
 'WRITE-IN61' 'WRITE-IN62' 'WRITE-IN63' 'WRITE-IN66' 'WRITE-IN69'
 'WRITE-IN70' 'WRITE-IN73' 'WRITE-IN77' 'WRITE-IN94' 'WRITE-IN99'
 'WRITE-IN103' 'WRITE-IN110' 'WRITE-IN122' 'WRITE-IN123' 'WRITE-IN124'
 'WRITE-IN137' 'WRITE-IN139' 'WRITE-IN152' 'WRITE-IN154' 'WRITE-IN156'
 'WRITE-IN159' 'WRITE-IN169' 'WRITE-IN170' 'WRITE-IN172' 'WRITE-IN177'
 'WRITE-IN182' 'WRITE-IN197' 'WRITE-IN198' 'WRITE-IN202' 'WRITE-IN203'
 'WRITE-IN222' 'WRITE-IN225' 'WRITE-IN232' 'WRITE-IN249' 'WRITE-IN252'
 'WRITE-IN256' 'WRITE-IN262' 'WRITE-IN263' 'WRITE-IN272' 'WRITE-IN280'
 'WRITE-IN304' 'WRITE-IN308' 'WRITE-IN309' 'WRITE-IN313' 'WRITE-IN325'
 'WRITE-IN326' 'WRITE-IN328' 'WRITE-IN330' 'WRITE-IN371' 'WRITE-IN373'
 'WRITE-IN375' 'WRITE-IN382' 'WRITE-IN390' 'WRITE-IN395' 'WRITE-IN404'
 'WRITE-IN408' 'WRITE-IN410' 'WRITE-IN411']


Replace non-candidates with '-'

In [8]:
data = data.replace('^[Ww][Rr][Ii][Tt][Ee].*[Ii][Nn].*$', 'writein', regex=True)
data = data.replace(['skipped', 'writein', 'Undeclared', 'Uncommitted', 'undervote'], '-')
data.head()

,rank1,rank2
0,Cindy Dyballa,-
1,Cindy Dyballa,-
2,Cindy Dyballa,-
3,Cindy Dyballa,-
4,Cindy Dyballa,-


Double Check Replace

In [9]:
print(data.rank1.unique())

['Cindy Dyballa' '-']


# Shift candidate entries


Convert to numpy matrix to allow specific index manipulation

In [10]:
data = data.to_numpy()

Ignore overvote instances and all rankings after

In [11]:
for i in range(data.shape[0]):
  overvoted = False
  for j in range(data.shape[1]):
    if data[i][j] == 'overvote':
      overvoted = True

    if overvoted:
      data[i][j] = '-'

Remove extra appearances of candidates in each row

In [12]:
for i in range(data.shape[0]):
  ballot = ["candidate entry"] * candNum
  for j in range(data.shape[1]):
    for b in range(len(ballot)):
      if data[i][j] != '-':
        if ballot[b] == "candidate entry":
          ballot[b] = data[i][j]
          break
        elif ballot[b] == data[i][j]:
          data[i][j] = '-'
          break

Shift Candidates to left until all non-candidates are to the right of all candidates

In [13]:
for i in range(np.size(data, 0)):
  for j in range(np.size(data, 1)-1):
    if data[i][j] == '-':
      k = j + 1
      while k < np.size(data, 1) and data[i][k] == '-':
        k += 1
      if k < np.size(data, 1):
        temp = data[i][j]
        data[i][j] = data[i][k]
        data[i][k] = temp

Return to pandas dataframe


In [14]:
data = pd.DataFrame(data, columns = attributes)
data.head()

,rank1,rank2
0,Cindy Dyballa,-
1,Cindy Dyballa,-
2,Cindy Dyballa,-
3,Cindy Dyballa,-
4,Cindy Dyballa,-


Remove extraneous row(s)

In [15]:
#data.rank4.unique()
#data = data.drop(columns = ['rank4'])
data.head()

,rank1,rank2
0,Cindy Dyballa,-
1,Cindy Dyballa,-
2,Cindy Dyballa,-
3,Cindy Dyballa,-
4,Cindy Dyballa,-


# Count duplicated rows

Return to numpy matrix

In [16]:
data = data.to_numpy()

Turn each row into a single entry in the numpy matrix

In [17]:
tempArray = np.empty((len(data), 1), dtype='<U512')

for i in range(len(data)):
  tempArray[i] = ("; ".join(data[i]))

Return to pandas dataframe/sort alphabetically

In [18]:
data = pd.DataFrame(tempArray, columns = ['Profile'])
data = data.sort_values(by= ['Profile'])
data.head()

,Profile
143,-; -
840,-; -
848,-; -
487,-; -
229,-; -


Find unique combinations

In [19]:
data = data.Profile.value_counts().sort_index()

# Convert Preference profile to csv

Find index names

In [20]:
indexNames = data.index.values

Determine number of entries per profile and number of profiles

In [21]:
entryCount = len(indexNames[0].split('; '))
profileCount = len(indexNames)

Turn data to numpy matrix

In [22]:
data = data.to_numpy()

Create new numpy matrix to hold preference profiles and count

In [23]:
preferenceProfile = np.empty((profileCount, entryCount + 1), dtype=object)

for i in range(len(data)):
  preferenceProfile[i][0] = data[i]
  names = indexNames[i].split('; ')
  for j in range(len(names)):
    preferenceProfile[i][j + 1] = names[j]

Covert preference profile to pandas dataframe

In [24]:
attributes = attributes.insert(0, 'Num. Voters')

preferenceProfile = pd.DataFrame(preferenceProfile, columns = attributes)
preferenceProfile.head()

,Num. Voters,rank1,rank2
0,45,-,-
1,1027,Cindy Dyballa,-


Save preferenceProfile as csv

In [25]:
preferenceProfile.to_csv('/content/drive/MyDrive/RCV Voting Method Research/Preference Profiles/'+filename+'NoWriteins.csv', encoding='utf-8', index=False)